In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [2]:
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

In [3]:
MAX_LEN = 40

In [4]:
bert_version = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_version)

In [5]:
train_df = pd.read_csv("/kaggle/input/quora-question-pairs/train.csv.zip")
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
submission = pd.read_csv("/kaggle/input/quora-question-pairs/sample_submission.csv.zip")
submission.head()

,test_id,is_duplicate
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [7]:
class BertDataSet:
    def __init__(self, q1, q2, target, tokenizer):
        self.q1 = q1
        self.q2 = q2
        self.target = target
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.q1)

    def __getitem__(self, item):
        q1 = str(self.q1[item])
        q2 = str(self.q2[item])
        
#         print(q1)
#         print(q2)
        q1 = " ".join(q1.split())
        q2 = " ".join(q2.split())
        
        inputs = self.tokenizer.encode_plus(
            q1,
            q2,
            add_special_tokens=True,
            padding='max_length',
            max_length=MAX_LEN,
            truncation=True
            
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
#         print(tokenizer.convert_ids_to_tokens(ids))
#         print(token_type_ids)
#         print(mask)
#         print("#################################")
        
        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(int(self.target[item]), dtype=torch.long) if self.target is not None else 0
        }
        

In [8]:
class BERTBaseUncased(nn.Module):
    def __init__(self, bert_path):
        super(BERTBaseUncased, self).__init__()
        self.bert_path = bert_path
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 1)

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(ids, attention_mask=mask,token_type_ids=token_type_ids)
        bo = self.bert_drop(o2) 
        return self.out(bo)

In [9]:
def loss_fn(outputs, targets):
    outputs = torch.squeeze(outputs)
    return nn.BCELoss()(nn.Sigmoid()(outputs), targets)

In [10]:
def train_loop(data_loader, model, optimizer, device, scheduler=None):
    it = 1
    total_loss = 0
    
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]
        
        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        outputs = model(ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
            
        if it % 500 == 0:
            print('Iter ', it, '| Avg Train Loss ', total_loss / 500)
            total_loss = 0
        
        it += 1
        

In [11]:
def eval_loop(data_loader, model, device):
    model.eval()
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]
        
        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()

In [12]:
def run(model, train_df, device):
    EPOCHS = 1
    BS = 128
    
    train_df, val_df = train_test_split(train_df, test_size=0.1)
    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)
    
    train_targets = train_df["is_duplicate"].values
    val_targets = val_df["is_duplicate"].values
    
    train_dataset = BertDataSet(
        q1=train_df["question1"].values,
        q2=train_df["question2"].values,
        target=train_targets,
        tokenizer=tokenizer
    )
    
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size = BS,
        shuffle=True
    )
    
    
    val_dataset = BertDataSet(
        q1=val_df["question1"].values,
        q2=val_df["question2"].values,
        target=val_targets,
        tokenizer=tokenizer
    )
    
    val_data_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=4,
        shuffle=True
    )
    
    lr = 3e-5
    num_training_steps = int(len(train_dataset) / BS * EPOCHS)
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )
    
    for epoch in range(EPOCHS):
        print("EPOCH: ", epoch)
        train_loop(train_data_loader, model, optimizer, device, scheduler)
    
    return model

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTBaseUncased(bert_version).to(device)
run(model, train_df, device)


EPOCH:  0
Iter  500 | Avg Train Loss  0.39913998752832414
Iter  1000 | Avg Train Loss  0.3166408561468124
Iter  1500 | Avg Train Loss  0.2923530434668064
Iter  2000 | Avg Train Loss  0.2792681589126587
Iter  2500 | Avg Train Loss  0.2693894906640053


BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [14]:
test_df = pd.read_csv("/kaggle/input/quora-question-pairs/test.csv")
test_df.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [15]:
def test(model, test_df, device):
    predictions = torch.empty(0).to(device, dtype=torch.float)
    
    test_dataset = BertDataSet(
        q1=test_df["question1"].values,
        q2=test_df["question2"].values,
        target=None,
        tokenizer=tokenizer
    )
    
    test_data_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=512
    )
    
    with torch.no_grad():
        model.eval()
        for bi, d in enumerate(test_data_loader):
            ids = d["ids"]
            mask = d["mask"]
            token_type_ids = d["token_type_ids"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)

            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            predictions = torch.cat((predictions, nn.Sigmoid()(outputs)))
    
    return predictions.cpu().numpy().squeeze()

In [16]:
predictions = test(model, test_df, device)
len(predictions)

2345796

In [17]:
test_df['is_duplicate'] = predictions

In [18]:
test_df[['test_id', 'is_duplicate']].to_csv('submission.csv', index=False)